In [ ]:
import pandas as pd
import folium
import os

# URL der GeoJSON-Datei
geojson_url = "https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson"

# Den Pfad zur CSV-Datei definieren (angenommen, sie liegt im 'Data'-Ordner deines Projekts)
csv_file_path = os.path.join(os.getcwd(), '..', 'Data', 'Dataset.csv')

# CSV-Datei einlesen
df = pd.read_csv(csv_file_path)

# Extrahiere Länder aus den Affiliations
def extract_countries(affiliation):
    try:
        affiliations = affiliation.split(";")
        countries = [a.split(",")[-1].strip() for a in affiliations]
        return countries
    except Exception:
        return []

df['Countries'] = df['Author Affiliations'].fillna('').apply(extract_countries)

# Explodieren der Länderliste, damit jedes Land eine Zeile wird
df_exploded = df.explode('Countries')

# Anzahl der Publikationen pro Land berechnen
country_counts = df_exploded['Countries'].value_counts().reset_index()
country_counts.columns = ['Country', 'Publication Count']

# Folium-Karte erstellen
m = folium.Map(location=[20, 0], zoom_start=2)

# Choropleth für die Publikationen
folium.Choropleth(
    geo_data=geojson_url,
    name="choropleth",
    data=country_counts,
    columns=["Country", "Publication Count"],
    key_on="feature.properties.ADMIN",
    fill_color="Blues",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Anzahl der Publikationen"
).add_to(m)

# Karte speichern oder anzeigen
m.save("map.html")
m
